# Merge SKB text files for all blocks of a powerline

In [13]:
import os
import sys
import glob
import fileinput
from pathlib import Path
sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata")
import SVK
import pandas as pd

# Merge SKB txt-files for all blocks of a powerline (LGXXX_Y) into one txt

LGs_info_path = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\Ledningar_2022_Kantträd_2023-02-06.txt"

prj_dir = Path(r"Z:\Projekt\Analys\ledningar")

def combine_blocks(row):
    LG = row[0]
    line = row[1]
    line_dir = prj_dir / LG / f"line_{line}"
    
    print(f"Doing {LG}_{line}")
    #block_dir = line_dir / "Kantträd" / "block"
    block_dir = os.path.join(line_dir, "kantträd", "block") #'Ledningar_2022_Kantträd_2023-02-06.txt')
    #combined_blocks_path = line_dir / "Kantträd" / f"SKB_raw.txt"
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge RBX files for all blocks into one
    SVK.merge_blocks(block_dir, "*_pt*.txt", combined_blocks_path) # DENNA SKILDE SIG FRÅN 2022 till 2021. Tidigare "*SKB_"

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
print(LGs_info)
LGs_info.apply(combine_blocks, axis=1)

       LG  line    Littera                         Namn  Strömtyp  Spänning  \
0   LG025     1     AL3 S8              Avgr till Åsele        10       220   
1   LG025     2     AL3 S5     Avgr Åsele-Stenkullafors        10       220   
2   LG025     3     AL3 S3               Gulsele-Hällby        10       220   
3   LG025     4     AL3 S4            Hällby-Avgr Åsele        10       220   
4   LG046     1     RL8 S3      Avgr Järkvissle-Hällsjö        10       220   
5   LG046     2     RL8 S9         Avgr till Järkvissle        10       220   
6   LG046     3     RL8 S7             Hällsjö-Söderala        10       220   
7   LG046     4     RL8 S2  Hölleforsen-Avgr Järkvissle        10       220   
8   LG046     5     CL3 S2            Nysäter-Vittersjö        10       400   
9   LG046     6       RL56          Stadsforsen-Hällsjö        10       220   
10  LG046     7     RL8 S1      Stadsforsen-Hölleforsen        10       220   
11  LG046     8     RL2 S1      Stadsforsen-Torpsham

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

# Calculate distances to wire for trees

In [1]:
# Prepare SKB feature class (point) with X,Y,Z,dZ,mZ,avt_fas,avst_hori
# (dZ = tree height, mZ = ground elevation at tree base)
# Point coordinates are X,Y,mZ
import sys
sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata\kantträd")
import os, SKB, time, arcpy
from time import gmtime, strftime
from pathlib import Path
import pandas as pd

# UPPDRAGSMAPP
prj_dir = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04")
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
LGs_info_path = os.path.join(prj_dir, 'Ledningar_2022_Kantträd_2023-02-06.txt')

# LOKALT
gdb = os.path.join(prj_dir, 'SVK_SKB_2023-02-06.gdb')
#gdb = Path(r"C:\Users\semnve\Desktop\Exempeldata_2021\SVK_2021_GIS\SVK_2021_GIS.gdb")

SKB_XYmZ_location = os.path.join(gdb, 'SKB_XYmZ')
SKB_XYZ_location = os.path.join(gdb, 'SKB_XYZ')
#input_dir = os.path.join(prj_dir, 'SKB_alla_LG')
#SKB_file_prefix = 'SKB_'
wires_fd_1 = os.path.join(gdb, 'Wires')
wires_fd_2 = os.path.join(gdb, 'Wires_incomplete')
wires_suffix = '_fas'
search_radius = 100 # How far from the point should Near3D search for wires
SKB_template = os.path.join(gdb, 'SKB_template')
atgardsar = 2024
leverantor = 'SWECO'
insamlingsmetod = 20
matosakerhet_p = 1000
matosakerhet_h = 1000
arcpy.env.overwriteOutput = True

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)


def compute_SKB_distances(row):
    LG = row[0]
    line = row[1]
    line_dir = prj_dir / "ledningar" / LG / f"line_{line}"
    SKB_dir = line_dir / "SKB"
    
    arcpy.env.overwriteOutput = True
    print(f"Starting with {LG}_{line}")

    #SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    #SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    #akuta_trad = f"{LG}_{line}_akuta"
    #ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    akuta_trad = f"{LG}_{line}_akuta"
    ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    SKB_XYmZ_path = os.path.join(SKB_XYmZ_location, SKB_XYmZ)
    SKB_XYZ_path = os.path.join(SKB_XYZ_location, SKB_XYZ)
    akuta_trad_path = os.path.join(SKB_XYZ_location, akuta_trad)
    ej_akuta_trad_path = os.path.join(SKB_XYZ_location, ej_akuta_trad)

    wires_1 = os.path.join(wires_fd_1, f"{LG}_{line}{wires_suffix}")
    wires_2 = os.path.join(wires_fd_2, f"{LG}_{line}{wires_suffix}")
    
    if arcpy.Exists(wires_1):
        wires = wires_1
        print(f"{LG}_{line} faslinor i Wires")
    elif arcpy.Exists(wires_2):
        wires = wires_2
        print(f"{LG}_{line} faslinor i Wires_incomplete")
    else:
        print(f"Inga faslinor för {LG}_{line}")
        return
    
    # Create an fc for X,Y,Z,dZ,mZ with mZ as Z coordinate
    #SKB_file_in = os.path.join(SKB_dir, f"{SKB_file_prefix}{LG}_{line}.txt")
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    SKB.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    print(f'{LG}_{line}_SKB_XYmZ created')

    # Call arcpy function Near3D to calculate distance from mZ to wire, 
    # on SKB_XYmZ
    SKB.dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    print(f'dist_mZ_wire calculated for {LG}_{line}')
    print(f'{strftime("%H:%M:%S", gmtime())}')

    # Create feature class for SKB where point XYZ is the XYZ for the tree tops
    SKB.create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)

    # Fill SKB_XYZ with values from SKB_XYmZ and set Z coordinate to Z (instead of mZ)
    LG_nr = int(LG[-3:])
    SKB.populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod,
                        matosakerhet_p, matosakerhet_h)
    print('SKB_XYZ populated')  

    # NEAR_DIST computed by Near3D, and then used as AvstHori in populate_SKB_XYZ,
    # does not seem to be the perpendicular distance to the wire, but rather the 
    # distance to the nearest point on the wire (due to sag, I guess)
    # So: Use NEAR analysis in 2D to create a new NEAR column, then use that one as AvstHori
    SKB.compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    print('Avstand fas computed')
          
    # Now replace old avst_hori with new, correct one from compute_horizontal_dist. 
    # Then remove the NEAR-fields.
    SKB.update_horizontal_dist(SKB_XYZ_path)
    print('Avstand horisontellt computed')
          
    # Pick out akuta trad (avstand fas <= 1)
    SKB.pick_out_akuta(SKB_XYZ_location, SKB_XYZ, akuta_trad)
    print('Akuta exported')
          
    # Remove duplicates from akuta_trad
    arcpy.DeleteIdentical_management(akuta_trad_path, ['Shape'], 0.05, 0.05)
    print('Duplicates removed')
    
    # Pick out ej akuta trad (avstand fas > 1)
    SKB.pick_out_ej_akuta(SKB_XYZ_location, SKB_XYZ, ej_akuta_trad)
    print('ej_akuta_trad exported')
    
    # Remove duplicates from ej_akuta_trad
    arcpy.DeleteIdentical_management(ej_akuta_trad_path, ['Shape'], 0.05, 0.05)
    print('Duplicates removed')
                
    print(f"Done with {LG}_{line}")
          
    xpath = os.path.abspath(SKB.__file__)
    print(xpath)
    
# Coordinate system
sr = sweref99TM()

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(compute_SKB_distances, axis=1)
    


Starting with LG025_1
LG025_1 faslinor i Wires

KeyboardInterrupt: 

In [1]:
##!!TESTVARIANT!!!##
# Prepare SKB feature class (point) with X,Y,Z,dZ,mZ,avt_fas,avst_hori
# (dZ = tree height, mZ = ground elevation at tree base)
# Point coordinates are X,Y,mZ
import sys
sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata\kantträd")
import os, SKB, time, arcpy
from time import gmtime, strftime
from pathlib import Path
import pandas as pd

# UPPDRAGSMAPP
prj_dir = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04")
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
LGs_info_path = os.path.join(prj_dir, 'Ledningar_2022_Kantträd_2023-02-06.txt')

# LOKALT
gdb = os.path.join(prj_dir, 'SVK_SKB_2023-02-06.gdb')
#gdb = Path(r"C:\Users\semnve\Desktop\Exempeldata_2021\SVK_2021_GIS\SVK_2021_GIS.gdb")

SKB_XYmZ_location = os.path.join(gdb, 'SKB_XYmZ')
SKB_XYZ_location = os.path.join(gdb, 'SKB_XYZ')
#input_dir = os.path.join(prj_dir, 'SKB_alla_LG')
#SKB_file_prefix = 'SKB_'
wires_fd_1 = os.path.join(gdb, 'Wires')
wires_fd_2 = os.path.join(gdb, 'Wires_incomplete')
wires_suffix = '_fas'
search_radius = 100 # How far from the point should Near3D search for wires
SKB_template = os.path.join(gdb, 'SKB_template')
atgardsar = 2024
leverantor = 'SWECO'
insamlingsmetod = 20
matosakerhet_p = 1000
matosakerhet_h = 1000
arcpy.env.overwriteOutput = True

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)


def compute_SKB_distances(row):
    LG = row[0]
    line = row[1]
    line_dir = prj_dir / "ledningar" / LG / f"line_{line}"
    SKB_dir = line_dir / "SKB"
    
    arcpy.env.overwriteOutput = True
    print(f"Starting with {LG}_{line}")

    #SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    #SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    #akuta_trad = f"{LG}_{line}_akuta"
    #ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    akuta_trad = f"{LG}_{line}_akuta"
    ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    SKB_XYmZ_path = os.path.join(SKB_XYmZ_location, SKB_XYmZ)
    SKB_XYZ_path = os.path.join(SKB_XYZ_location, SKB_XYZ)
    akuta_trad_path = os.path.join(SKB_XYZ_location, akuta_trad)
    ej_akuta_trad_path = os.path.join(SKB_XYZ_location, ej_akuta_trad)

    wires_1 = os.path.join(wires_fd_1, f"{LG}_{line}{wires_suffix}")
    wires_2 = os.path.join(wires_fd_2, f"{LG}_{line}{wires_suffix}")
    
    if arcpy.Exists(wires_1):
        wires = wires_1
        print(f"{LG}_{line} faslinor i Wires")
    elif arcpy.Exists(wires_2):
        wires = wires_2
        print(f"{LG}_{line} faslinor i Wires_incomplete")
    else:
        print(f"Inga faslinor för {LG}_{line}")
        return
    
    # Create an fc for X,Y,Z,dZ,mZ with mZ as Z coordinate
    #SKB_file_in = os.path.join(SKB_dir, f"{SKB_file_prefix}{LG}_{line}.txt")
    #SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    #SKB.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    #print(f'{LG}_{line}_SKB_XYmZ created')

    SKB.update_z_coordinate(os.path.join(SKB_XYmZ_path, SKB_XYmZ))
    
    
# Coordinate system
sr = sweref99TM()

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(compute_SKB_distances, axis=1)
    

Starting with LG025_1
LG025_1 faslinor i Wires


0    None
dtype: object

In [2]:
# ANVÄNDS ENBART OM MAN SKA TA BORT SÅDANA DATASET SOM REDAN FINNS
import arcpy
import os

gdb = r"C:\Users\semnve\Desktop\Exempeldata_2021\SVK_2021_GIS\SVK_2021_GIS_lokal.gdb"
fd_SKB_XYmZ = os.path.join(gdb, "SKB_XYmZ")
fd_SKB_XYZ = os.path.join(gdb, "SKB_XYZ")

lines = ["LG006_2"] #"LG006_2", "LG037_5", "LG061_1", "LG113_2", "LG117_1", "LG123_1","LG123_2","LG137_1"]

for line in lines:
    SKB_XYmZ = os.path.join(gdb, fd_SKB_XYmZ, line+"_SKB_XYmZ")
    SKB_XYZ = os.path.join(gdb, fd_SKB_XYZ, line+"_SKB_XYZ")
    arcpy.management.Delete(SKB_XYmZ)
    print(f"{line}_SKB_XYmZ deleted")
    arcpy.management.Delete(SKB_XYZ)
    print(f"{line}_SKB_XYZ deleted")
    akuta = os.path.join(gdb, fd_SKB_XYZ, line+"_akuta")
    ej_akuta = os.path.join(gdb, fd_SKB_XYZ, line+"_ej_akuta")
    arcpy.management.Delete(akuta)
    arcpy.management.Delete(ej_akuta)


LG006_2_SKB_XYmZ deleted
LG006_2_SKB_XYZ deleted


# Add scan date and accuracy

## RUN! Scandates

In [8]:
scandates = {"LG006_1":[2021,7,9],
             "LG006_2":[2021,7,9],
             "LG006_3":[2021,7,9],
             "LG006_4":[2021,7,9],
             "LG006_5":[2021,7,6],
             "LG006_6":[2021,7,9],
             "LG006_7":[2021,7,9],
             "LG009_1":[2021,7,9],
             "LG011_1":[2021,7,6],
             "LG011_2":[2021,7,6],
             "LG013_1":[2021,7,9],
             "LG013_2":[2021,7,9],
             "LG013_3":[2021,7,9],
             "LG013_4":[2021,7,9],
             "LG014_1":[2021,7,9],
             "LG014_2":[2021,7,9],
             "LG014_3":[2021,7,9],
             "LG014_4":[2021,7,9],
             "LG014_5":[2021,7,9],
             "LG014_6":[2021,7,9],
             "LG016_1":[2021,7,6],
             "LG016_2":[2021,7,6],
             "LG018_1":[2021,7,6],
             "LG018_2":[2021,7,6],
             "LG018_3":[2021,7,6],
             "LG026_1":[2021,7,9],
             "LG026_2":[2021,7,9],
             "LG026_3":[2021,7,9],
             "LG026_4":[2021,7,9],
             "LG027_1":[2021,7,9],
             "LG027_2":[2021,7,9],
             "LG027_3":[2021,7,9],
             "LG037_1":[2021,7,12],
             "LG037_2":[2021,7,10],
             "LG037_3":[2021,7,12],
             "LG037_4":[2021,7,10],
             "LG037_5":[2021,7,9],
             "LG042_1":[2021,7,13],
             "LG043_1":[2021,7,13],
             "LG043_2":[2021,7,13],
             "LG043_3":[2021,7,13],
             "LG043_4":[2021,7,13],
             "LG044_1":[2021,7,13],
             "LG044_2":[2021,7,13],
             "LG044_3":[2021,7,13],
             "LG044_4":[2021,7,13],
             "LG044_5":[2021,7,13],
             "LG047_1":[2021,7,6],
             "LG047_2":[2021,7,6],
             "LG049_1":[2021,7,6],
             "LG049_2":[2021,7,6],
             "LG049_3":[2021,7,6],
             "LG049_4":[2021,7,6],
             "LG049_5":[2021,7,6],
             "LG060_1":[2021,7,13],
             "LG060_2":[2021,7,13],
             "LG061_1":[2021,7,13],
             "LG068_1":[2021,7,14],
             "LG068_2":[2021,7,14],
             "LG068_3":[2021,7,14],
             "LG068_4":[2021,7,14],
             "LG068_5":[2021,7,14],
             "LG068_6":[2021,7,14],
             "LG068_7":[2021,7,14],
             "LG068_8":[2021,7,14],
             "LG068_9":[2021,7,14],
             "LG068_10":[2021,7,14],
             "LG068_11":[2021,7,14],
             "LG079_1":[2021,7,14],
             "LG079_2":[2021,7,14],
             "LG082_1":[2021,7,14],
             "LG082_2":[2021,7,14],
             "LG082_3":[2021,7,14],
             "LG082_4":[2021,7,14],
             "LG082_5":[2021,7,14],
             "LG082_6":[2021,7,14],
             "LG083_1":[2021,7,14],
             "LG083_2":[2021,7,14],
             "LG083_3":[2021,7,14],
             "LG083_4":[2021,7,14],
             "LG083_5":[2021,7,14],
             "LG083_6":[2021,7,14],
             "LG083_7":[2021,7,14],
             "LG083_8":[2021,7,14],
             "LG083_9":[2021,7,14],
             "LG083_10":[2021,7,14],
             "LG083_11":[2021,7,14],
             "LG083_12":[2021,7,14],
             "LG083_13":[2021,7,14],
             "LG093_1":[2021,7,14],
             "LG093_2":[2021,7,14],
             "LG098_1":[2021,7,14],
             "LG098_2":[2021,7,14],
             "LG103_1":[2021,6,6],
             "LG107_1":[2021,6,6],
             "LG107_2":[2021,6,6],
             "LG109_1":[2021,7,5],
             "LG109_2":[2021,7,5],
             "LG109_3":[2021,7,5],
             "LG113_1":[2021,6,9],
             "LG113_2":[2021,6,9],
             "LG115_1":[2021,6,9],
             "LG115_2":[2021,6,9],
             "LG115_3":[2021,6,9],
             "LG117_1":[2021,6,9],
             "LG122_1":[2021,6,5],
             "LG122_2":[2021,6,5],
             "LG122_3":[2021,6,5],
             "LG122_4":[2021,6,5],
             "LG123_1":[2021,6,7],
             "LG123_2":[2021,6,2],
             "LG123_3":[2021,7,12],
             "LG123_4":[2021,7,12],
             "LG123_5":[2021,7,12],
             "LG123_6":[2021,7,12],
             "LG124_1":[2021,6,2],
             "LG124_2":[2021,6,2],
             "LG124_3":[2021,6,2],
             "LG124_4":[2021,6,2],
             "LG124_5":[2021,6,2],
             "LG130_1":[2021,6,2],
             "LG130_2":[2021,6,2],
             "LG132_1":[2021,6,2],
             "LG132_2":[2021,6,2],
             "LG132_3":[2021,6,2],
             "LG132_4":[2021,6,2],
             "LG137_1":[2021,5,29],
             "LG137_2":[2021,5,29],
             "LG138_1":[2021,5,29],
             "LG138_2":[2021,5,29],
             "LG138_3":[2021,5,29],
             "LG138_4":[2021,5,29],
             "LG139_1":[2021,5,29],
             "LG139_2":[2021,5,29],
             "LG140_1":[2021,5,29],
             "LG143_1":[2021,5,29],
             "LG143_2":[2021,5,29],
             "LG410_1":[2021,7,5],
             "LG500_1":[2021,7,5],
             "LG500_2":[2021,7,5],
             "LG661_1":[2021,7,5],
             "LG661_2":[2021,7,5],
             "LG710_1":[2021,6,1],
             "LG710_2":[2021,6,1],
             "LG730_1":[2021,6,9]}


## RUN! Mätosäkerheter

In [9]:
matosakerheter_h = {"LG006_1":10,
                    "LG006_2":10,
                    "LG006_3":10,
                    "LG006_4":10,
                    "LG006_5":10,
                    "LG006_6":10,
                    "LG006_7":10,
                    "LG009_1":10,
                    "LG011_1":10,
                    "LG011_2":10,
                    "LG013_1":10,
                    "LG013_2":10,
                    "LG013_3":10,
                    "LG013_4":10,
                    "LG014_1":10,
                    "LG014_2":10,
                    "LG014_3":10,
                    "LG014_4":10,
                    "LG014_5":10,
                    "LG014_6":10,
                    "LG016_1":10,
                    "LG016_2":10,
                    "LG018_1":10,
                    "LG018_2":10,
                    "LG018_3":10,
                    "LG026_1":10,
                    "LG026_2":10,
                    "LG026_3":10,
                    "LG026_4":10,
                    "LG027_1":10,
                    "LG027_2":10,
                    "LG027_3":10,
                    "LG037_1":10,
                    "LG037_2":10,
                    "LG037_3":10,
                    "LG037_4":10,
                    "LG037_5":10,
                    "LG042_1":10,
                    "LG043_1":10,
                    "LG043_2":10,
                    "LG043_3":10,
                    "LG043_4":10,
                    "LG044_1":10,
                    "LG044_2":10,
                    "LG044_3":10,
                    "LG044_4":10,
                    "LG044_5":10,
                    "LG047_1":10,
                    "LG047_2":10,
                    "LG049_1":10,
                    "LG049_2":10,
                    "LG049_3":10,
                    "LG049_4":10,
                    "LG049_5":10,
                    "LG060_1":10,
                    "LG060_2":10,
                    "LG061_1":10,
                    "LG068_1":10,
                    "LG068_2":10,
                    "LG068_3":10,
                    "LG068_4":10,
                    "LG068_5":10,
                    "LG068_6":10,
                    "LG068_7":10,
                    "LG068_8":10,
                    "LG068_9":10,
                    "LG068_10":10,
                    "LG068_11":10,
                    "LG079_1":10,
                    "LG079_2":10,
                    "LG082_1":10,
                    "LG082_2":10,
                    "LG082_3":10,
                    "LG082_4":10,
                    "LG082_5":10,
                    "LG082_6":10,
                    "LG083_1":10,
                    "LG083_2":10,
                    "LG083_3":10,
                    "LG083_4":10,
                    "LG083_5":10,
                    "LG083_6":10,
                    "LG083_7":10,
                    "LG083_8":10,
                    "LG083_9":10,
                    "LG083_10":10,
                    "LG083_11":10,
                    "LG083_12":10,
                    "LG083_13":10,
                    "LG093_1":10,
                    "LG093_2":10,
                    "LG098_1":10,
                    "LG098_2":10,
                    "LG103_1":10,
                    "LG107_1":10,
                    "LG107_2":10,
                    "LG109_1":10,
                    "LG109_2":10,
                    "LG109_3":10,
                    "LG113_1":10,
                    "LG113_2":10,
                    "LG115_1":10,
                    "LG115_2":10,
                    "LG115_3":10,
                    "LG117_1":10,
                    "LG122_1":10,
                    "LG122_2":10,
                    "LG122_3":10,
                    "LG122_4":10,
                    "LG123_1":10,
                    "LG123_2":10,
                    "LG123_3":10,
                    "LG123_4":10,
                    "LG123_5":10,
                    "LG123_6":10,
                    "LG124_1":10,
                    "LG124_2":10,
                    "LG124_3":10,
                    "LG124_4":10,
                    "LG124_5":10,
                    "LG130_1":10,
                    "LG130_2":10,
                    "LG132_1":10,
                    "LG132_2":10,
                    "LG132_3":10,
                    "LG132_4":10,
                    "LG137_1":10,
                    "LG137_2":10,
                    "LG138_1":10,
                    "LG138_2":10,
                    "LG138_3":10,
                    "LG138_4":10,
                    "LG139_1":10,
                    "LG139_2":10,
                    "LG140_1":10,
                    "LG143_1":10,
                    "LG143_2":10,
                    "LG410_1":10,
                    "LG500_1":10,
                    "LG500_2":10,
                    "LG661_1":10,
                    "LG661_2":10,
                    "LG710_1":10,
                    "LG710_2":10,
                    "LG730_1":10}

## Scan dates and accuracies to Akuta

In [10]:
##!!DENNA BEHÖVS INTE LÄNGRE!!##
#AVVAKTA MED DENNA

# Add scan date (Registreringsdatum/RGDTM) to the akuta
import datetime, arcpy, os, SVK
from pathlib import Path
import pandas as pd

prj_dir = Path(r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021")

# Uppdragsmapp
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"

# Lokalt
gdb = Path(r"C:\Users\SEKNNO\Desktop\TMP_hemmajobb\SVK_2021_lokal\SVK_2021_GIS_lokal.gdb")

akuta_location = os.path.join(gdb, 'SKB_XYZ')

LGs_info_path = prj_dir / "ledningar_2021_sample.txt"

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'
                    
def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")
    
    akuta_path = os.path.join(akuta_location, f"{LG}_{line}_akuta")
    
    scandate = datetime.date(scandates[LG_line][0], scandates[LG_line][1], scandates[LG_line][2])
    matosakerhet_h = matosakerheter_h[LG_line]
    
    with arcpy.da.UpdateCursor(akuta_path, [date_field, matosakerhet_h_field]) as u_cursor:
        for row in u_cursor:
            row[0] = scandate
            row[1] = matosakerhet_h
            u_cursor.updateRow(row)
    print(scandate)

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(add_date_and_accuracy, axis=1)
    

Doing LG006_2
2021-07-09
Doing LG037_5
2021-07-09
Doing LG061_1
2021-07-13
Doing LG107_1
2021-06-06
Doing LG107_2
2021-06-06
Doing LG113_2
2021-06-09
Doing LG117_1
2021-06-09
Doing LG123_1
2021-06-07
Doing LG123_2
2021-06-02
Doing LG130_1
2021-06-02
Doing LG137_1
2021-05-29


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
dtype: object

In [3]:
#ANVÄNDS

import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

LGs_info_file = "Ledningar_2022_Kantträd_2023-02-06.txt"
prj_dir = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04"
SKB_XYZ_fd = os.path.join(prj_dir,"SVK_SKB_2023-02-06.gdb","SKB_XYZ")                                                                                            

LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    # if arcpy.Exists(f"{LG}_{line}_XYZ_fc"):
    for fieldname in fieldnames:
        
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
        print(f"Altering fields for {LG}_{line}{fieldname}")
        path_joined = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
        field_names = [f.name for f in arcpy.ListFields(path_joined)]
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if "LEDNINGSGATA" in field_names:    
            arcpy.management.DeleteField(
                path_joined,
                "LEDNINGSGATA")
        if "INSAMLINGSMETOD" in field_names:    
            arcpy.management.DeleteField(
                path_joined,
                "INSAMLINGSMETOD")

    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG025_1_SKB_XYZ
Altering fields for LG025_1_akuta
Altering fields for LG025_1_ej_akuta
Altering fields for LG025_2_SKB_XYZ
Altering fields for LG025_2_akuta
Altering fields for LG025_2_ej_akuta
Altering fields for LG025_3_SKB_XYZ
Altering fields for LG025_3_akuta
Altering fields for LG025_3_ej_akuta
Altering fields for LG025_4_SKB_XYZ
Altering fields for LG025_4_akuta
Altering fields for LG025_4_ej_akuta
Altering fields for LG046_1_SKB_XYZ
Altering fields for LG046_1_akuta
Altering fields for LG046_1_ej_akuta
Altering fields for LG046_2_SKB_XYZ
Altering fields for LG046_2_akuta
Altering fields for LG046_2_ej_akuta
Altering fields for LG046_3_SKB_XYZ
Altering fields for LG046_3_akuta
Altering fields for LG046_3_ej_akuta
Altering fields for LG046_4_SKB_XYZ
Altering fields for LG046_4_akuta
Altering fields for LG046_4_ej_akuta
Altering fields for LG046_5_SKB_XYZ
Altering fields for LG046_5_akuta
Altering fields for LG046_5_ej_akuta
Altering fields for LG046_6_SKB_XYZ
A

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

In [4]:
# Add scan date (Registreringsdatum/RGDTM) to the akuta
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

gdb = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb")
LGs_info_file = "Ledningar_2022_Kantträd_2023-02-06.txt"
prj_dir = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04"

LGs_info_path = os.path.join(prj_dir, LGs_info_file)
cvd_LEDNINGSGATA_path = r"C:\Users\semnve\Desktop\Exempeldata_2021\cvd_LEDNINGSGATA.txt"
cvd_DATE_path = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\cvd_DATE.txt"

SKB_location = os.path.join(gdb, 'SKB_XYZ')

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    LG_date = cvd_DATE[LG.join("_00" + str(line))]

    #SKB_path = os.path.join(SKB_location, f"{LG}_{line}_SKB_XYZ")

    #fc_SKB = f"{LG}_{line}_SKB_XYZ"
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")

    #if arcpy.Exists(SKB_path):
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True

        # Creating actual fields in the geodatabase and calculating values
        arcpy.management.AddField(fc_SKB, "LITTERA", "TEXT")
        arcpy.management.CalculateField(fc_SKB, "LITTERA", "".join(("'", littera, "'")))
        arcpy.management.AddField(fc_SKB, "LEDNINGSGATA", "LONG")
        arcpy.management.CalculateField(fc_SKB, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(fc_SKB, "RGDTM", "DATE")
        arcpy.management.CalculateField(fc_SKB, "RGDTM", "".join(("'", LG_date, "'")))
        arcpy.management.AddField(fc_SKB, "Ursprung", "TEXT")
        arcpy.management.CalculateField(fc_SKB, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(fc_SKB, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(fc_SKB, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(fc_SKB, "Insamlingsmetod", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(fc_SKB, "Matosakerhet_Plan", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(fc_SKB, "Matosakerhet_Hojd", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Matosakerhet_Hojd", int(matosak_hojd))
        #field = ['AVSTAND_FAS']
        #arcpy.management.CalculateField(fc_SKB, field, round(field[0], 1))
        #arcpy.CalculateField_management(SKB_path, "AVSTAND_FAS", '"{:.1f}".format(round(!AVSTAND_FAS!, 1))', "PYTHON_9.3")

        dataset = SKB_location
        sr = arcpy.SpatialReference("SWEREF99_TM")
        arcpy.DefineProjection_management(dataset, sr)
        # Assigning domains to fields
        arcpy.management.AssignDomainToField(
            fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(
            fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")

    print("Klart")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
    len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {df_cvd_DATE.LG[i].join(
    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(add_date_and_accuracy, axis=1)


Doing LG025_1
Klart
Doing LG025_2
Klart
Doing LG025_3
Klart
Doing LG025_4
Klart
Doing LG046_1
Klart
Doing LG046_2
Klart
Doing LG046_3
Klart
Doing LG046_4
Klart
Doing LG046_5
Klart
Doing LG046_6
Klart
Doing LG046_7
Klart
Doing LG046_8
Klart
Doing LG058_1
Klart
Doing LG071_1
Klart
Doing LG071_2
Klart
Doing LG071_3
Klart
Doing LG071_4
Klart
Doing LG500_1
Klart
Doing LG500_2
Klart


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

In [5]:
import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

LGs_info_file = "Ledningar_2022_Kantträd_2023-02-06.txt"
prj_dir = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04"
SKB_XYZ_fd = os.path.join(prj_dir,"SVK_SKB_2023-02-06.gdb","SKB_XYZ")                                                                                            

LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]
            if "AVSTAND_HORISONTELLT" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['AVSTAND_HORISONTELLT']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "AVSTAND_FAS" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_FAS",
                    "AVSTAND_FAS",
                    "AVSTAND_FAS")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['AVSTAND_FAS']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "DELTA_HOJD" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "DELTA_HOJD",
                    "DELTA_HOJD",
                    "DELTA_HOJD")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['DELTA_HOJD']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "RGDTM" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "RGDTM",
                    "RGDTM",
                    "RGDTM")

            if "MATOSAKERHET_PLAN" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "MATOSAKERHET_PLAN",
                    "Matosakerhet_Plan",
                    "Matosakerhet_Plan")

            if "MATOSAKERHET_HOJD" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "MATOSAKERHET_HOJD",
                    "Matosakerhet_Hojd",
                    "Matosakerhet_Hojd")

            if "LEDNINGSGATA" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "LEDNINGSGATA",
                    "LEDNINGSGATA",
                    "LEDNINGSGATA")

            #if "GIS_ID" in field_names:    
            #        arcpy.management.DeleteField(
            #            path_joined,
            #            "GIS_ID")
            #if "MAX_TILLVAXT" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "MAX_TILLVAXT")
            #if "created_user" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "created_user")
            #if "created_date" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "created_date")
            #if "last_edited_user" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "last_edited_user")
            #if "last_edited_date" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "last_edited_date")
            #if "ATGARDSAR" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "ATGARDSAR")
            #if "LEVERANTOR" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "LEVERANTOR")
            #if "LEVERANTORS_DATUM" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "LEVERANTORS_DATUM")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG025_1_SKB_XYZ
Altering fields for LG025_1_akuta
Altering fields for LG025_1_ej_akuta
No RBX for LG025_1
Altering fields for LG025_2_SKB_XYZ
Altering fields for LG025_2_akuta
Altering fields for LG025_2_ej_akuta
No RBX for LG025_2
Altering fields for LG025_3_SKB_XYZ
Altering fields for LG025_3_akuta
Altering fields for LG025_3_ej_akuta
No RBX for LG025_3
Altering fields for LG025_4_SKB_XYZ
Altering fields for LG025_4_akuta
Altering fields for LG025_4_ej_akuta
No RBX for LG025_4
Altering fields for LG046_1_SKB_XYZ
Altering fields for LG046_1_akuta
Altering fields for LG046_1_ej_akuta
No RBX for LG046_1
Altering fields for LG046_2_SKB_XYZ
Altering fields for LG046_2_akuta
Altering fields for LG046_2_ej_akuta
No RBX for LG046_2
Altering fields for LG046_3_SKB_XYZ
Altering fields for LG046_3_akuta
Altering fields for LG046_3_ej_akuta
No RBX for LG046_3
Altering fields for LG046_4_SKB_XYZ
Altering fields for LG046_4_akuta
Altering fields for LG046_4_ej_akuta
No RBX for 

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

## Scan dates and accuracies to Traffpunkter

In [11]:
##!!!DEN HÄR BEHÖVS INTE LÄNGRE!!##

# Add scan date (Registreringsdatum/RGDTM) to the akuta
import datetime, arcpy, os, SVK
from pathlib import Path
import pandas as pd

prj_dir = Path(r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021")

# Uppdragsmapp
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"

# Lokalt
gdb = Path(r"C:\Users\SEKNNO\Desktop\TMP_hemmajobb\SVK_2021_lokal\SVK_2021_GIS_lokal.gdb")

traffpunkter_location = os.path.join(gdb, 'SKB_XYZ')

LGs_info_path = prj_dir / "ledningar_2021_sample.txt"

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'

def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")
    
    traffpunkter_path = os.path.join(traffpunkter_location, f"{LG}_{line}_ej_akuta")
    
    scandate = datetime.date(scandates[LG_line][0], scandates[LG_line][1], scandates[LG_line][2])
    matosakerhet_h = matosakerheter_h[LG_line]
    
    with arcpy.da.UpdateCursor(traffpunkter_path, [date_field, matosakerhet_h_field]) as u_cursor:
        for row in u_cursor:
            row[0] = scandate
            row[1] = matosakerhet_h
            u_cursor.updateRow(row)
    print(scandate)

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(add_date_and_accuracy, axis=1)



Doing LG006_2
2021-07-09
Doing LG037_5
2021-07-09
Doing LG061_1
2021-07-13
Doing LG107_1
2021-06-06
Doing LG107_2
2021-06-06
Doing LG113_2
2021-06-09
Doing LG117_1
2021-06-09
Doing LG123_1
2021-06-07
Doing LG123_2
2021-06-02
Doing LG130_1
2021-06-02
Doing LG137_1
2021-05-29


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
dtype: object

# Merge Akuta to one

In [1]:
# Merge all Akuta to one
import datetime, arcpy, os
from pathlib import Path
import pandas as pd

#### REMEMBER TO CHANGE THIS ONE TO AVOID OVERWRITING PREVIOUS ONE ####
akuta_merged_name = "akuta_2023_02_06"

LGs_info_name = "Ledningar_2022_Kantträd_2023-02-06.txt"

prj_dir = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04")
# UPPDRAGSMAPP
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
# LOKALT
gdb = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb")
akuta_location = os.path.join(gdb, 'SKB_XYZ')
akuta_merged_location = os.path.join(gdb, 'akuta')
akuta_merged_path = os.path.join(akuta_merged_location, akuta_merged_name)
LGs_info_path = prj_dir / LGs_info_name

# LÄGG TILL ATT OM INTE "akut"-DATASET FINNS SÅ SKA DEN SKAPAS

def to_akuta_list(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    print(f"Adding {LG}_{line} to list")
    
    akuta_path = os.path.join(akuta_location, f"{LG}_{line}_akuta")

    akuta_list.append(akuta_path)
    
    print(f"Done adding {LG}_{line} to list")

    
akuta_list = []

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(to_akuta_list, axis=1)

print("Doing merge")
arcpy.Merge_management(akuta_list, akuta_merged_path)


Adding LG025_1 to list
Done adding LG025_1 to list
Adding LG025_2 to list
Done adding LG025_2 to list
Adding LG025_3 to list
Done adding LG025_3 to list
Adding LG025_4 to list
Done adding LG025_4 to list
Adding LG046_1 to list
Done adding LG046_1 to list
Adding LG046_2 to list
Done adding LG046_2 to list
Adding LG046_3 to list
Done adding LG046_3 to list
Adding LG046_4 to list
Done adding LG046_4 to list
Adding LG046_5 to list
Done adding LG046_5 to list
Adding LG046_6 to list
Done adding LG046_6 to list
Adding LG046_7 to list
Done adding LG046_7 to list
Adding LG046_8 to list
Done adding LG046_8 to list
Adding LG058_1 to list
Done adding LG058_1 to list
Adding LG071_1 to list
Done adding LG071_1 to list
Adding LG071_2 to list
Done adding LG071_2 to list
Adding LG071_3 to list
Done adding LG071_3 to list
Adding LG071_4 to list
Done adding LG071_4 to list
Adding LG500_1 to list
Done adding LG500_1 to list
Adding LG500_2 to list
Done adding LG500_2 to list
Doing merge


<Result 'C:\\Users\\semnve\\Desktop\\Kantträd 2023-02-04\\SVK_SKB_2023-02-06.gdb\\akuta\\akuta_2023_02_06'>

# Merge ej_akuta to one

In [2]:
# Merge all ej_akuta to one
import datetime, arcpy, os
from pathlib import Path
import pandas as pd

#### REMEMBER TO CHANGE THIS ONE TO AVOID OVERWRITING PREVIOUS ONE ####
ej_akuta_merged_name = "ej_akuta_2023_02_06"

LGs_info_name = "Ledningar_2022_Kantträd_2023-02-06.txt"

prj_dir = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04")
# UPPDRAGSMAPP
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
# LOKALT
#gdb = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb")
gdb = os.path.join(prj_dir, "SVK_SKB_2023-02-06.gdb")
ej_akuta_location = os.path.join(gdb, 'SKB_XYZ')
ej_akuta_merged_location = os.path.join(gdb, 'ej_akuta')
ej_akuta_merged_path = os.path.join(ej_akuta_merged_location, ej_akuta_merged_name)
LGs_info_path = prj_dir / LGs_info_name

def to_ej_akuta_list(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    print(f"Adding {LG}_{line} to list")
    
    ej_akuta_path = os.path.join(ej_akuta_location, f"{LG}_{line}_ej_akuta")

    ej_akuta_list.append(ej_akuta_path)
    
    print(f"Done adding {LG}_{line} to list")

    
ej_akuta_list = []

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(to_ej_akuta_list, axis=1)

print("Doing merge")
arcpy.Merge_management(ej_akuta_list, ej_akuta_merged_path)

#print(ej_akuta_list)
#print(ej_akuta_merged_path)

Adding LG025_1 to list
Done adding LG025_1 to list
Adding LG025_2 to list
Done adding LG025_2 to list
Adding LG025_3 to list
Done adding LG025_3 to list
Adding LG025_4 to list
Done adding LG025_4 to list
Adding LG046_1 to list
Done adding LG046_1 to list
Adding LG046_2 to list
Done adding LG046_2 to list
Adding LG046_3 to list
Done adding LG046_3 to list
Adding LG046_4 to list
Done adding LG046_4 to list
Adding LG046_5 to list
Done adding LG046_5 to list
Adding LG046_6 to list
Done adding LG046_6 to list
Adding LG046_7 to list
Done adding LG046_7 to list
Adding LG046_8 to list
Done adding LG046_8 to list
Adding LG058_1 to list
Done adding LG058_1 to list
Adding LG071_1 to list
Done adding LG071_1 to list
Adding LG071_2 to list
Done adding LG071_2 to list
Adding LG071_3 to list
Done adding LG071_3 to list
Adding LG071_4 to list
Done adding LG071_4 to list
Adding LG500_1 to list
Done adding LG500_1 to list
Adding LG500_2 to list
Done adding LG500_2 to list
Doing merge


<Result 'C:\\Users\\semnve\\Desktop\\Kantträd 2023-02-04\\SVK_SKB_2023-02-06.gdb\\ej_akuta\\ej_akuta_2023_02_06'>

# Klipp träffpunkter

In [3]:
import arcpy
import os
import pandas as pd


version = "2023_02_06"

gdb = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb"
LGs_info_name = "Ledningar_2022_Kantträd_2023-02-06.txt"
fc_ledningsgator = os.path.join(gdb, "Gator2021")
fc_stationsomraden = os.path.join(gdb, "STATIONSOMRADE")

arcpy.env.overwriteOutput = True

#fd_akuta = os.path.join(gdb, "akuta")
fd_ej_akuta = os.path.join(gdb, "ej_akuta")
fd_traffpunkter = os.path.join(gdb, "traffpunkter")

#fc_akuta = os.path.join(fd_akuta, f"akuta_{version}")
fc_ej_akuta = os.path.join(fd_ej_akuta, f"ej_akuta_{version}")
fc_traffpunkter = os.path.join(fd_traffpunkter, f"traffpunkter_{version}")
tmp_layer_traffpunkter = os.path.join(gdb, "tmp_layer_traffpunkter")
#tmp_layer_ledningsgator = os.path.join(gdb, "tmp_layer_ledningsgator")
#tmp_layer_stationsomraden = os.path.join(gdb, "tmp_layer_stationsomraden")

# LÄGG TILL I SKRIPT ATT DET KIKAR OM DET FINNS ETT DATASET SOM HETER "traffpunkter" ANNARS SKAPA DET

# Gör temporära lager att arbeta med
# Ta för säkerhets skull bort eventuella existerande lager
arcpy.management.Delete(tmp_layer_traffpunkter)
arcpy.management.MakeFeatureLayer(fc_ej_akuta, tmp_layer_traffpunkter)
#arcpy.management.Delete(tmp_layer_ledningsgator)
#arcpy.management.MakeFeatureLayer(fc_ledningsgator, tmp_layer_ledningsgator)
#arcpy.management.Delete(tmp_layer_stationsomraden)
#arcpy.management.MakeFeatureLayer(fc_ej_akuta, tmp_layer_stationsomraden)
#
# Steg 1
# Inom fastbredd och stationsområden: ta bort alla inom 7 m horisontellt avstånd från fas
#
# 1.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 1.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 1.3 Av de valda, behåll punkter med avst_hori < 7 m
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                        selection_type = "SUBSET_SELECTION", 
                                        where_clause = '"AVSTAND_HORISONTELLT" < 7')
# 1.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Steg 2
# Inom fastbredd och stationsområden: ta bort alla med trädhöjd <= 4.5 m och avst_fas >= 5 m
#
# 2.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 2.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 2.3 Av de valda, behåll punkter med trädhöjd <= 4.5 OCH avst_fas >= 5
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                       selection_type = "SUBSET_SELECTION",
                                       where_clause = '"DELTA_HOJD" <= 4.5 AND "AVSTAND_FAS" >= 5')
# 2.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Kopiera ej_akuta till traffpunkter
arcpy.management.CopyFeatures(tmp_layer_traffpunkter, fc_traffpunkter)

# Ta bort det temporära lagret
arcpy.management.Delete(tmp_layer_traffpunkter)


<Result 'true'>

In [4]:
# Rensar rådata från onödig information


import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

LGs_info_file = "Ledningar_2022_Kantträd_2023-02-06.txt"
prj_dir = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04"
SKB_XYZ_fd = os.path.join(prj_dir,"SVK_SKB_2023-02-06.gdb","SKB_XYZ")                                                                                            

LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, fc_SKB)
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]

            if "GIS_ID" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "GIS_ID")
            if "MAX_TILLVAXT" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "MAX_TILLVAXT")
            if "created_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_user")
            if "created_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_date")
            if "last_edited_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_user")
            if "last_edited_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_date")
            if "ATGARDSAR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "ATGARDSAR")
            if "LEVERANTOR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTOR")
            if "LEVERANTORS_DATUM" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTORS_DATUM")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG025_1_SKB_XYZ
Altering fields for LG025_1_akuta
Altering fields for LG025_1_ej_akuta
No RBX for LG025_1
Altering fields for LG025_2_SKB_XYZ
Altering fields for LG025_2_akuta
Altering fields for LG025_2_ej_akuta
No RBX for LG025_2
Altering fields for LG025_3_SKB_XYZ
Altering fields for LG025_3_akuta
Altering fields for LG025_3_ej_akuta
No RBX for LG025_3
Altering fields for LG025_4_SKB_XYZ
Altering fields for LG025_4_akuta
Altering fields for LG025_4_ej_akuta
No RBX for LG025_4
Altering fields for LG046_1_SKB_XYZ
Altering fields for LG046_1_akuta
Altering fields for LG046_1_ej_akuta
No RBX for LG046_1
Altering fields for LG046_2_SKB_XYZ
Altering fields for LG046_2_akuta
Altering fields for LG046_2_ej_akuta
No RBX for LG046_2
Altering fields for LG046_3_SKB_XYZ
Altering fields for LG046_3_akuta
Altering fields for LG046_3_ej_akuta
No RBX for LG046_3
Altering fields for LG046_4_SKB_XYZ
Altering fields for LG046_4_akuta
Altering fields for LG046_4_ej_akuta
No RBX for 

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

In [4]:
# Rensar rådata från onödig information


import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

LGs_info_file = "Ledningar_2022_Kantträd_2023-02-06.txt"
prj_dir = r"C:\Users\semnve\Desktop\Kantträd 2023-02-04"
#SKB_XYZ_fd = os.path.join(prj_dir,"SVK_2021_GIS","SVK_2021_GIS.gdb","SKB_XYZ")                                                                                            

LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    
    datasetnames = ["akuta", "ej_akuta", "traffpunkter"]
    fieldnames = ["2023_02_06"]
    
    for dataset in datasetnames:
        print(dataset)
        for fieldname in fieldnames:
            fc_SKB = f"{dataset}_{fieldname}"
            if "akuta" in dataset:
                SKB_location = os.path.join(prj_dir,"SVK_SKB_2023-02-06.gdb",f"{dataset}")
            else:
                SKB_location = os.path.join(prj_dir,"SVK_SKB_2023-02-06.gdb",f"{dataset}")
            print(SKB_location)
            SKB_path = os.path.join(SKB_location, fc_SKB)
 
            # Activating workspace
            arcpy.env.workspace = SKB_path
            arcpy.env.overwriteOutput = True

            if arcpy.Exists(fc_SKB):
                SKB_XYZ_fc = fc_SKB
                print(f"Altering fields for {dataset}_{fieldname}")
                path_joined = os.path.join(SKB_location, f'{dataset}_{fieldname}')
                field_names = [f.name for f in arcpy.ListFields(SKB_path)]

                if "GIS_ID" in field_names:    
                        arcpy.management.DeleteField(
                            path_joined,
                            "GIS_ID")
                if "MAX_TILLVAXT" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "MAX_TILLVAXT")
                if "created_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_user")
                if "created_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_date")
                if "last_edited_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_user")
                if "last_edited_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_date")
                if "ATGARDSAR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "ATGARDSAR")
                if "LEVERANTOR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTOR")
                if "LEVERANTORS_DATUM" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTORS_DATUM")
                
                dataset = SKB_location
                sr = arcpy.SpatialReference("SWEREF99_TM")
                arcpy.DefineProjection_management(dataset, sr)
                # Assigning domains to fields
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
    #else:
    #    print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

akuta
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\akuta
Altering fields for akuta_2023_02_06
ej_akuta
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\ej_akuta
Altering fields for ej_akuta_2023_02_06
traffpunkter
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\traffpunkter
Altering fields for traffpunkter_2023_02_06
akuta
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\akuta
Altering fields for akuta_2023_02_06
ej_akuta
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\ej_akuta
Altering fields for ej_akuta_2023_02_06
traffpunkter
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\traffpunkter
Altering fields for traffpunkter_2023_02_06
akuta
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\akuta
Altering fields for akuta_2023_02_06
ej_akuta
C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb\ej_akuta
Altering fields for ej_akuta_2023_02_06
tra

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object

# Update leverantörsdatum

In [6]:
##!! DENNA BEHÖVS INTE LÄNGRE!!##


# Update leverantorsdatum

# Add scan date to RBX
import os, arcpy, datetime, SVK
from pathlib import Path

feature_classes = ["traffpunkter_15mars", "alla_traffpunkter_2021_pre15marsOK", "akuta_15mars", "alla_akuta_2021_pre15marsOK"]

#prj_dir = Path(r"Z:\26664\6605189\000\Projekt\Utleveranser\Skannat_2021\Leverans_2021-12-15")
#gdb = prj_dir / "Leverans_2021-12-15.gdb"

gdb = r"C:\Users\SEKNNO\Desktop\TMP_hemmajobb\SVK_2021_lokal\Alla_kantträd_2021.gdb"

for feature_class in feature_classes:
    print(f"Updating leverantörsdatum for {feature_class}")
    feature_class_path = os.path.join(gdb, feature_class)
    
    update_fields = ['LEVERANTORS_DATUM']
    
    with arcpy.da.UpdateCursor(feature_class_path, update_fields) as u_cursor:
        for row in u_cursor:
            leverantorsdatum = datetime.date.today()
            row[0] = leverantorsdatum
            u_cursor.updateRow(row)
            #print(row)


Updating leverantörsdatum for traffpunkter_15mars
Updating leverantörsdatum for alla_traffpunkter_2021_pre15marsOK
Updating leverantörsdatum for akuta_15mars
Updating leverantörsdatum for alla_akuta_2021_pre15marsOK


# Add LGXXX field

In [5]:
##!!DENNA BEHÖVS INTE LÄNGRE!!

import arcpy
import os
#from pathlib import Path
import pandas as pd


# PATHS AND NAMES
LG_codes_path = r"Z:\26664\6605189\000\Arbetsmapp_analys\cvd_LG.txt"
gdb = r"C:\Users\SEKNNO\Desktop\TMP_hemmajobb\SVK_2021_lokal\Alla_kantträd_2021.gdb"
fc_names = ["traffpunkter_15mars", "alla_traffpunkter_2021_pre15marsOK", "akuta_15mars", "alla_akuta_2021_pre15marsOK"]


# FIELDS
LG_code_field = "LEDNINGSGATA"
LGXXX_field = "LGXXX"
update_fields = [LG_code_field, LGXXX_field]


# FUNCIONS
def create_LG_name(LG):
    return("LG" + str(LG).zfill(3))

def create_LG_code_dict(LG_codes_path):
    # Läs in LG-koder och gör en dictionary {kod:'LGXXX'}
    LG_codes_ = pd.read_csv(LG_codes_path, sep="\t", index_col=0)
    LG_codes_["LG_name"] = LG_codes_["1"].map(create_LG_name)
    LG_codes_.drop("1", axis=1, inplace=True)
    LG_codes = LG_codes_.to_dict()["LG_name"]
    return(LG_codes)

def add_LGXXX_field(fc):
    arcpy.AddField_management(in_table=fc,
                             field_name=LGXXX_field,
                             field_type="TEXT")

def update_LGXXX_field(fc):
    with arcpy.da.UpdateCursor(fc, update_fields) as u_cursor:
            for row in u_cursor:
                # row[0] is LG_code
                # LG_codes[row[0]] is the LGXXX
                # row[1] is empty, LGXXX (LG_codes[row[0]]) should be written there           
                row[1] = LG_codes[row[0]]
                u_cursor.updateRow(row)

                
# MAIN
LG_codes = create_LG_code_dict(LG_codes_path)

for fc_name in fc_names:
    print(f"Doing {fc_name}")
    fc = os.path.join(gdb, fc_name)
    add_LGXXX_field(fc)
    print(f"\tLGXXX field added")
    update_LGXXX_field(fc)
    print(f"\tLGXXX field updated")



Doing alla_traffpunkter_2021_pre15marsOK
	LGXXX field added
	LGXXX field updated
Doing akuta_15mars
	LGXXX field added
	LGXXX field updated
Doing alla_akuta_2021_pre15marsOK
	LGXXX field added
	LGXXX field updated


In [4]:
LG_codes

{20: 'LG002',
 30: 'LG003',
 40: 'LG004',
 50: 'LG005',
 60: 'LG006',
 70: 'LG007',
 80: 'LG008',
 90: 'LG009',
 100: 'LG010',
 110: 'LG011',
 120: 'LG012',
 130: 'LG013',
 140: 'LG014',
 150: 'LG015',
 160: 'LG016',
 170: 'LG017',
 180: 'LG018',
 190: 'LG019',
 200: 'LG020',
 210: 'LG021',
 220: 'LG022',
 230: 'LG023',
 240: 'LG024',
 250: 'LG025',
 260: 'LG026',
 270: 'LG027',
 280: 'LG028',
 290: 'LG029',
 300: 'LG030',
 310: 'LG031',
 320: 'LG032',
 330: 'LG033',
 340: 'LG034',
 350: 'LG035',
 360: 'LG036',
 370: 'LG037',
 380: 'LG038',
 390: 'LG039',
 400: 'LG040',
 410: 'LG041',
 420: 'LG042',
 430: 'LG043',
 440: 'LG044',
 450: 'LG045',
 460: 'LG046',
 470: 'LG047',
 480: 'LG048',
 490: 'LG049',
 500: 'LG050',
 510: 'LG051',
 520: 'LG052',
 530: 'LG053',
 540: 'LG054',
 550: 'LG055',
 560: 'LG056',
 570: 'LG057',
 580: 'LG058',
 590: 'LG059',
 600: 'LG060',
 610: 'LG061',
 620: 'LG062',
 630: 'LG063',
 640: 'LG064',
 650: 'LG065',
 660: 'LG066',
 670: 'LG067',
 680: 'LG068',
 69

# Delete stuff

In [6]:
import arcpy
import os

gdb = r"C:\Users\SEKNNO\Desktop\TMP_hemmajobb\SVK_2021_lokal\SVK_2021_GIS_lokal.gdb"

fd_SKB_XYZ = "SKB_XYZ"
fd_SKB_XYmZ = "SKB_XYmZ"

#ledningar = ["LG006_2", "LG037_5", "LG061_1", "LG107_1", "LG107_2", 
#             "LG113_2", "LG117_1", "LG123_1", "LG123_2", "LG130_1", "LG137_1"]
print("kör uppdaterad")
ledningar = []

for ledning in ledningar:
    XYZ_path = os.path.join(gdb, fd_SKB_XYZ, ledning+"_SKB_XYZ")
    akuta_path = os.path.join(gdb, fd_SKB_XYZ, ledning+"_akuta")
    ej_akuta_path = os.path.join(gdb, fd_SKB_XYZ, ledning+"_ej_akuta")
    XYmZ_path = os.path.join(gdb, fd_SKB_XYmZ, ledning+"_SKB_XYmZ")
    arcpy.management.Delete(XYZ_path)
    arcpy.management.Delete(akuta_path)
    arcpy.management.Delete(ej_akuta_path)
    arcpy.management.Delete(XYmZ_path)
    print(f"{ledning} done")

kör uppdaterad
LG130_1 done
LG037_5 done
LG061_1 done
LG107_1 done


# Old garbage

In [ ]:
import arcpy
import os
folder = r'Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2019\SVK_2019_maps\Leverans_20200519.gdb'
FCs = ['SkogenAktTrdMm', 'SkogenRBX', 'SkogenTraffpunkter', 'Wires_2019']

for fc in FCs:
    fc_path = os.path.join(folder, fc)
    arcpy.Delete_management(fc_path)


In [ ]:
## Create buffers around the LG:s where intrangsersatt mark does not exist, according to agreement about fixed width
## in email from Marcus Liljeberg 2020-05-15
## Buffer width is half the fixed width of the corridor
#
#import arcpy
#import SVK
#import os
#
#data_dir = r'Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2019'
#
#corridor_width = {38:44, 39:44, 45:32, 65:32, 72:46, 85:46,
#                 96:32, 105:32, 112:32}
#
#LGs = {38:[1], 39:{1}, 45:[1, 2], 65:[5, 6], 72:[2],
#      85:[3], 96:[1, 2, 3, 4], 105:[1, 2], 112:[2, 3, 4, 7]}
#
#fc_intrangsersatt_mark_fast_bredd = r'C:\Users\seknno\Desktop\SVK_digitalisering\Etapp_4_del_1_skannat_2019\SVK_Resultat_2020-04-21.gdb\ledningsgator_fast_bredd'
#
#to_merge = []
#
#for LG in LGs:
#    LG_name = SVK.LG_str(LG)
#    print(LG_name)
#    lines = LGs[LG]
#    buffer_dist = corridor_width[LG]/2
#
#    for line in lines:
#        #line_dir = os.path.join(data_dir, LG_name, 'line_'+str(line))
#        #mittlinje = os.path.join(line_dir, 'shp_mittlinje', LG_name+'_'+str(line)+'.shp')
#        #arcpy.Buffer_analysis(mittlinje, 
#        #                      os.path.join(r'C:\Users\seknno\Desktop\SVK_digitalisering\Etapp_4_del_1_skannat_2019\SVK_Resultat_2020-04-21.gdb', LG_name+'_line_'+str(line)+'_fast_bredd'),
#        #                      buffer_dist, 
#        #                     '#', '#', '#', '#', '#')
#        to_merge.append(os.path.join(r'C:\Users\seknno\Desktop\SVK_digitalisering\Etapp_4_del_1_skannat_2019\SVK_Resultat_2020-04-21.gdb', LG_name+'_line_'+str(line)+'_fast_bredd'))
#        #arcpy.Delete_management(os.path.join(r'C:\Users\seknno\Desktop\SVK_digitalisering\Etapp_4_del_1_skannat_2019\SVK_Resultat_2020-04-21.gdb', LG_name+'_line_'+str(line)+'_fast_bredd'))
#
#arcpy.Merge_management(to_merge, fc_intrangsersatt_mark_fast_bredd)        
#print(to_merge)   
#                              